# Effective Threshold Estimation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Optimizers import compute_effective_threshold

## Plot 1: Full Bloch sphere view

This plot provides an overview of the infidelities across the Bloch sphere.

### Scan

First, perform a scan over different error angles, applying $U(\theta, \phi)$, attempting QEC (maximum rounds: 1), and computing fidelities at each step.

In [ ]:
interior_points, fidelities = compute_effective_threshold(
    None, **steane_code,
    min_theta=0, max_theta=np.pi, n_theta=16,
    min_phi=0, max_phi=np.pi*2, n_phi=16,
)

### Plot

Get a list of angles and infidelities for a single specified `initial_state`. We choose between plotting all data points and only plotting those that are interior points (i.e. they do not exceed the effective threshold). We can then plot the infidelities against a Bloch sphere.

In [ ]:
initial_state = "0"
plot_all = True

if plot_all:
    angles_list = list(fidelities[initial_state].keys())
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    infidelities = 1-np.array(list(fidelities[initial_state].values()))
else:
    angles_list = list(interior_points.values())[0]
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = [fidelities[initial_state][angle] for angle in angles_list]
    infidelities = 1-np.array(fidelities_list)

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(projection="3d")
ax.set_aspect("equal")

R = 1
u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:30j]
x = R*np.cos(v) * np.sin(u)
y = R*np.sin(v) * np.sin(u)
z = R*np.cos(u)
ax.plot_surface(x, y, z, cmap=plt.cm.bone)

R *= 1.1
xs = R*np.cos(phis) * np.sin(thetas)
ys = R*np.sin(phis) * np.sin(thetas)
zs = R*np.cos(thetas)
scatter = ax.scatter(xs, ys, zs, c=infidelities, cmap=plt.cm.viridis, s=30)

ax.set_title("Infidelity vs. error angle")

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

plt.colorbar(scatter)

plt.show()

## Plot 2: Fine $\ket{0}$ pole scan

In [ ]:
interior_points, fidelities = compute_effective_threshold(
    None, **steane_code,
    min_theta=0, max_theta=np.pi/6, n_theta=32,
    min_phi=0, max_phi=np.pi*2, n_phi=32,
)

In [ ]:
initial_state = "0"
plot_all = False

if plot_all:
    angles_list = list(fidelities[initial_state].keys())
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    infidelities = 1-np.array(list(fidelities[initial_state].values()))
else:
    angles_list = list(interior_points.values())[0]
    
    thetas = np.array([angles[0] for angles in angles_list])
    phis = np.array([angles[1] for angles in angles_list])
    fidelities_list = [fidelities[initial_state][angle] for angle in angles_list]
    infidelities = 1-np.array(fidelities_list)

In [ ]:
fig, ax = plt.subplots(dpi=120)
ax.set_aspect("equal")

xs = R*np.cos(phis) * np.sin(thetas)
ys = R*np.sin(phis) * np.sin(thetas)
# zs = R*np.cos(thetas)
scatter = ax.scatter(xs, ys, c=infidelities, cmap=plt.cm.viridis, s=64)

ax.set_title("Infidelity vs. error angle")

ax.set_xlabel("X")
ax.set_ylabel("Y")

plt.colorbar(scatter)

plt.show()